In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, Dense, Flatten, Dropout, BatchNormalization, ReLU, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import operator
import cv2
import sys, os
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [ ]:
dct = {'P' : 0, 'F' : 1, 'G' : 2, 'K': 3, 'L':4, 'Y':5, 'nothing':6}

In [ ]:
path = "/content/drive/My Drive/SOAI/Ass 3/new_data"
data_folders = sorted(os.listdir(path))
X = []
Y = []
for i in data_folders:
  count = 0
  for j in os.listdir(path + "/" + i):
    x = cv2.imread(path + "/" + i + "/" + j)
    x = cv2.resize(x, (64, 64))
    #x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
    #_, x = cv2.threshold(x, 120, 255, cv2.THRESH_BINARY)
    #x = np.expand_dims(x, axis = -1)
    #x = np.concatenate((x,x,x), axis = -1)

    X.append(x)
    y = np.zeros((7))
    y[dct[i]] = 1
    Y.append(y)
    print(i,j)
    count+=1
    print(count)
    if(count >= 2000):
      break
X = np.asarray(X)
Y = np.asarray(Y)
print(X.shape, Y[0:10])

Streaming output truncated to the last 5000 lines.
1506
Y Y20.jpg
1507
Y Y2023.jpg
1508
Y Y2047.jpg
1509
Y Y1167.jpg
1510
Y Y1589.jpg
1511
Y Y1978.jpg
1512
Y Y2057.jpg
1513
Y Y1632.jpg
1514
Y Y1180.jpg
1515
Y Y1068.jpg
1516
Y Y14.jpg
1517
Y Y1442.jpg
1518
Y Y1562.jpg
1519
Y Y1453.jpg
1520
Y Y1516.jpg
1521
Y Y1611.jpg
1522
Y Y205.jpg
1523
Y Y1166.jpg
1524
Y Y1120.jpg
1525
Y Y1606.jpg
1526
Y Y1643.jpg
1527
Y Y148.jpg
1528
Y Y1419.jpg
1529
Y Y1126.jpg
1530
Y Y1408.jpg
1531
Y Y1670.jpg
1532
Y Y1653.jpg
1533
Y Y1590.jpg
1534
Y Y1618.jpg
1535
Y Y1997.jpg
1536
Y Y1497.jpg
1537
Y Y1983.jpg
1538
Y Y1683.jpg
1539
Y Y2702.jpg
1540
Y Y2442.jpg
1541
Y Y1890.jpg
1542
Y Y238.jpg
1543
Y Y2644.jpg
1544
Y Y2693.jpg
1545
Y Y2793.jpg
1546
Y Y2303.jpg
1547
Y Y2690.jpg
1548
Y Y2687.jpg
1549
Y Y2388.jpg
1550
Y Y2441.jpg
1551
Y Y2215.jpg
1552
Y Y2427.jpg
1553
Y Y1939.jpg
1554
Y Y2345.jpg
1555
Y Y1704.jpg
1556
Y Y1935.jpg
1557
Y Y2443.jpg
1558
Y Y2446.jpg
1559
Y Y2794.jpg
1560
Y Y2207.jpg
1561
Y Y2319.jpg
1562

In [ ]:
'''np.save('/content/drive/My Drive/SOAI/Ass 3/X_2000_color_3.npy', X)
np.save('/content/drive/My Drive/SOAI/Ass 3/Y_2000_color_3.npy', Y)'''
X = np.load('/content/drive/My Drive/SOAI/Ass 3/X_2000_color_3.npy')
Y = np.load('/content/drive/My Drive/SOAI/Ass 3/Y_2000_color_3.npy')

In [ ]:

'''Preparing the train/test data and training the model'''
X = X/127 -1
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=0.2, shuffle = True, random_state=42)
print(X_train.shape, X_test.shape, Y_test[0:10])

(11200, 64, 64, 3) (2800, 64, 64, 3) [[0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]


In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/SOAI/Ass 3/m_3_3.h5')

In [ ]:
vgg = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(64,64,3),
    pooling=None,
)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:

def model_fn():
  input = Input(shape = (64,64,3))
  x = vgg(input)
  # Flattening the layers
  x = Flatten()(x)

  # Adding a fully connected layer
  x = Dense(units=512, activation='relu')(x)
  x = Dense(units=100, activation='relu')(x)
  x = Dense(units=7, activation='softmax')(x)
  m = Model(input, x)
  return m

In [ ]:

model = model_fn()



In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               51300     
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 707       
Total params: 15,815,783
Trainable params: 15,815,783
Non-trainable params: 0
__________________________________________

In [ ]:

for i in range(2):
  model.fit(X_train, Y_train, batch_size = int(Y_train.shape[0]/10),epochs = 100, validation_data = (X_test, Y_test))
  model.save("/content/drive/My Drive/SOAI/Ass 3/m_3_3.h5")


Epoch 1/100
10/10 [==============================] - 12s 1s/step - loss: 2.2741 - accuracy: 0.1525 - val_loss: 1.8851 - val_accuracy: 0.2954
Epoch 2/100
10/10 [==============================] - 12s 1s/step - loss: 1.8569 - accuracy: 0.2371 - val_loss: 1.6439 - val_accuracy: 0.3314
Epoch 3/100
10/10 [==============================] - 12s 1s/step - loss: 1.4332 - accuracy: 0.3637 - val_loss: 1.4554 - val_accuracy: 0.3557
Epoch 4/100
10/10 [==============================] - 12s 1s/step - loss: 1.2306 - accuracy: 0.4434 - val_loss: 1.0006 - val_accuracy: 0.5021
Epoch 5/100
10/10 [==============================] - 12s 1s/step - loss: 0.9654 - accuracy: 0.5763 - val_loss: 0.8882 - val_accuracy: 0.6311
Epoch 6/100
10/10 [==============================] - 12s 1s/step - loss: 0.8573 - accuracy: 0.6263 - val_loss: 0.7396 - val_accuracy: 0.6900
Epoch 7/100
10/10 [==============================] - 12s 1s/step - loss: 1.1145 - accuracy: 0.5133 - val_loss: 0.8482 - val_accuracy: 0.6500
Epoch 8/100
1

KeyboardInterrupt: ignored

In [ ]:
model.save("/content/drive/My Drive/SOAI/Ass 3/m_3_3.h5")